<a href="https://colab.research.google.com/github/derek-shing/DS-Unit2-Sprint5-Kaggle/blob/master/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Data

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce

In [0]:
!pip install category_encoders

In [4]:
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_features = pd.read_csv('/content/gdrive/My Drive/train_features.csv')
train_labels = pd.read_csv('/content/gdrive/My Drive/train_labels.csv')
test_features = pd.read_csv('/content/gdrive/My Drive/test_features.csv')
submission = pd.read_csv('/content/gdrive/My Drive/sample_submission.csv')
construction_year_median = train_features[train_features['construction_year']!=0]['construction_year'].median()
install_less_than_300 =  train_features['installer'].value_counts()[train_features['installer'].value_counts()<300].index
install_less_than_300= list(install_less_than_300)
funder_less_than_100 =  train_features['funder'].value_counts()[train_features['funder'].value_counts()<100].index
funder_less_than_100= list(funder_less_than_100)


In [0]:
high_cardinality =list(train_features.select_dtypes(exclude=np.number).nunique()[train_features.select_dtypes(exclude=np.number).nunique()>50].index)

In [37]:
train_features.shape, train_labels.shape, test_features.shape

((59400, 40), (59400, 2), (14358, 40))

In [0]:
train_features.funder.value_counts()

In [0]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [0]:
train_features.head().T

In [0]:
y_train = train_labels['status_group']
X_train = train_features.drop(columns='id')
X_test  = test_features.drop(columns='id')

In [0]:
train_labels.head()

In [280]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((44550, 39), (14850, 39), (44550,), (14850,))

In [0]:
y_train.value_counts(normalize=True)

In [0]:
X_train.isnull().sum()

# Study Relation

In [0]:
X_y =X_train.copy()

In [0]:
X_y['y']=y_train

In [174]:
X_y.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'y'],
      dtype='object')

In [179]:
pd.pivot_table(X_y,index=["permit"],values=["y"],aggfunc=[np.sum])

,sum
,y
permit,
False,14346
True,34471


In [178]:
X_y.public_meeting.isnull().sum()

2475

In [275]:
X_y.funder.value_counts()

Government Of Tanzania            6845
Danida                            2302
Hesawa                            1647
Rwssp                             1051
World Bank                        1014
World Vision                       958
Kkkt                               955
Unicef                             794
Tasaf                              664
District Council                   627
Private Individual                 619
Dhv                                609
Dwsp                               606
0                                  580
Norad                              575
Germany Republi                    464
Tcrs                               458
Ministry Of Water                  430
Water                              429
Dwe                                380
Hifab                              353
Netherlands                        353
Adb                                331
Lga                                330
Amref                              316
Fini Water               

#Modeling

In [0]:
def wrangle_3(X):
  X = X.copy()
  low_cardinality = ['basin','region',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'source_type','quantity_group',
       'source_class', 'waterpoint_type', 'waterpoint_type_group']
  
  #install_less_than_300 =  X['installer'].value_counts()[X['installer'].value_counts()<300].index
  #install_less_than_300= list(install_less_than_300)
  #print(install_less_than_300)
  X[X['installer'].isin(install_less_than_300)]['installer']='Other'
  X['installer']=X['installer'].fillna('Other')
  
  X[X['funder'].isin(funder_less_than_100)]['funder']='Other'
  X['funder']=X['funder'].fillna('Other')
  
  #X.installer.value_counts()
  
  #X=pd.get_dummies(X,columns=low_cardinality)
  quality_group_map = {'unknown':0,'salty':1,'milky':2,'colored':3,'fluoride':4,'good':5}
  X['quality_group'] =  X['quality_group'].map(quality_group_map)
  #X['quality_group_square']=X['quality_group']**2
  
  X['permit']=X['permit'].fillna(False)
  X['permit']=X['permit'].replace({True:1,False:0})
 
  X['public_meeting']=X['public_meeting'].fillna(False)
  X['public_meeting']=X['public_meeting'].replace({True:1,False:0})  
  
   
  high_cardinality =['date_recorded','wpt_name','subvillage','ward','scheme_name']
  X=X.drop(high_cardinality, axis=1)
  X=X.drop(['scheme_management'], axis=1)
  
  subclass_feature=['extraction_type_class','extraction_type_group']
  #X=X.drop(subclass_feature,axis=1)
  
  X['construction_year'].replace({0:construction_year_median})
  #X['Age_square']=(2018-X['construction_year'])**2
  #print(X.isnull().sum())
  #print(X.columns)
  return X


In [281]:
X_train_numeric = wrangle_3(X_train)
X_val_numeric = wrangle_3(X_val)
X_test_numeric = wrangle_3(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [282]:
X_train_numeric.shape, X_val_numeric.shape,X_test_numeric.shape

((44550, 33), (14850, 33), (14358, 33))

In [16]:
set(X_test_numeric.columns)-set(X_val_numeric.columns)

set()

In [0]:
X_train_numeric.quality_group.value_counts()

In [0]:
X_train_numeric.isnull().sum()

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    #LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=10000)
    #KNeighborsClassifier()
    RandomForestClassifier()
)


In [0]:
X_train_q=pd.get_dummies(X_train,columns=['quality_group'])

In [0]:
X_try=X_train_q[['quality_group_colored',
       'quality_group_fluoride', 'quality_group_good', 'quality_group_milky',
       'quality_group_salty', 'quality_group_unknown', 'y']]

In [0]:
X_try.y.value_counts()
X_try.y=X_try.y.map(status_map)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
status_map={'non functional':0,'functional needs repair':1,'functional':2}

In [0]:
y_train = y_train.replace(status_map)

In [0]:
y_train

In [286]:
m = pipeline.fit(X_train_numeric,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
X_train.quality_group.value_counts()

In [0]:
X_train_numeric.select_dtypes(exclude = np.number).nunique()

In [0]:
X_train_numeric.extraction_type.value_counts()

In [0]:
  X_train_numeric.extraction_type_group.value_counts()

In [0]:
  X_train_numeric.extraction_type_class.value_counts()

#Old Transform

In [0]:
def wrangle(X):
  X = X.copy()
  low_cardinality = ['basin','region',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'source_type','quantity_group',
       'source_class', 'waterpoint_type', 'waterpoint_type_group']
  
  X=pd.get_dummies(X,columns=low_cardinality)
  quality_group_map = {'unknown':0,'salty':1,'milky':2,'colored':3,'fluoride':4,'good':5}
  X['quality_group'] =  X['quality_group'].map(quality_group_map)
  X['quality_group_square']=X['quality_group']**2
  
  X = X.select_dtypes(np.number)
  
  
  return X

def wrangle_OneCode(X):
  X = X.copy()
  low_cardinality = ['basin','region',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'source_type','quantity_group',
       'source_class', 'waterpoint_type', 'waterpoint_type_group']
  
  #install_less_than_300 =  X['installer'].value_counts()[X['installer'].value_counts()<300].index
  #install_less_than_300= list(install_less_than_300)
  #print(install_less_than_300)
  X[X['installer'].isin(install_less_than_300)]['installer']='Other'
  X['installer']=X['installer'].fillna('Other')
  
  #X.installer.value_counts()
  
  #X=pd.get_dummies(X,columns=low_cardinality)
  quality_group_map = {'unknown':0,'salty':1,'milky':2,'colored':3,'fluoride':4,'good':5}
  X['quality_group'] =  X['quality_group'].map(quality_group_map)
  #X['quality_group_square']=X['quality_group']**2
   
  high_cardinality =['date_recorded','funder','wpt_name','subvillage','ward','scheme_name']
  X=X.drop(high_cardinality, axis=1)
  X=X.drop(['public_meeting','scheme_management','permit'], axis=1)
  
  subclass_feature=['extraction_type_class','extraction_type_group']
  #X=X.drop(subclass_feature,axis=1)
  
  #X['construction_year'].replace({0:construction_year_mean})
  #X['Age_square']=(2018-X['construction_year'])**2
  #print(X.isnull().sum())
  #print(X.columns)
  return X


# Fit and predict and val

In [0]:
X_train.quality_group.value_counts()

In [0]:
list(X_train.select_dtypes(exclude=np.number).nunique()[X_train.select_dtypes(exclude=np.number).nunique()>100].index)

In [0]:
X_train['construction_year'].replace({0:X_train['construction_year'].mean()})

In [0]:
construction_year_mean = X_train[X_train['construction_year']!=0]['construction_year'].mean()

In [0]:
wrangle_OneCode(X_train).isnull().sum()

In [287]:
y_pred=m.predict(X_val_numeric)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [0]:
y_val=y_val.map(status_map)

In [271]:
y_pred

array([2, 2, 0, ..., 1, 2, 2])

In [0]:
y_val

In [289]:
accuracy_score(y_val,y_pred)

0.7856565656565656

In [0]:
pd.Series(y_pred).value_counts()

2    8233
0    5522
1     603
dtype: int64

#Testing

In [290]:
y_pred=m.predict(X_test_numeric)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [0]:
y_pred = pd.Series(y_pred)

In [0]:
status_map_r = {0:'non functional',1:'functional needs repair',2:'functional'}

In [0]:
y_pred = y_pred.replace(status_map_r)

In [298]:
y_pred.value_counts()

functional                 7899
non functional             5668
functional needs repair     791
dtype: int64

In [0]:
submission['status_group']=y_pred

In [0]:
from google.colab import files

submission.to_csv('test_labels.csv',index=False)
files.download('test_labels.csv')